In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
from getcpuinfo import cpuinfo
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from sklearn.metrics import accuracy_score

In [3]:
def getTrainedLSTMModel(maxLengthParam):
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,maxLengthParam,1), return_sequences = True))
    
    model.add(Dense(50, activation = 'tanh'))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM(25,return_sequences = False))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("LSTMBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("LSTMBestWeights.hdf5")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;
    
    
def getTrained3OuptutModel(maxLengthParam):
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Flatten(batch_input_shape = (None,maxLengthParam,1)))
    
    model.add(Dense(50, activation = "relu"))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(25, activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Dense(12,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(6,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("3OutputDNNBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("3OutputDNNBestWeights.hdf5")
        
        print("Returning Trained Model")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;

In [17]:
def appendToList(oldList,newList,maxLength):
    
    #append to the new entries
    oldList = np.insert(oldList,0,np.array([newList]),axis = 0)
    
    #Remove oldest row if there are more than 50 entries
    if(oldList.shape[0]>maxLength):
        oldList = oldList[:49,:]

    return oldList

def getPredictionsForOneTimeStep(model,usageList,maxLengthParam):
    
    currUsageList = cpuinfo()
    
    updatedList = appendToList(usageList,currUsageList,maxLengthParam)
    
    predictions = []
    
    predDict = {0:0,1:1,2:-1}
    
    for i in range(updatedList.shape[1]):
        
        if updatedList.shape[0]==maxLengthParam:
            
            modelOutput = model.predict(np.expand_dims(np.expand_dims(updatedList[:,i],axis = 1),axis = 0))

            modelPrediction = predDict[np.argmax(modelOutput)]

            predictions.append(modelPrediction)
        
        else:
            
            predictions.append(0)
    
    return predictions, updatedList

#overProvisionedList = 

def getRightOutputs(usageList,minThreshold, maxThreshold):
    
    currList = usageList[0,:]
    print(currList)
    
    target = []
    
    for i in range(usageList.shape[1]):
        if currList[i] > maxThreshold:# Scale Up
            #print("MaxThres ",maxThreshold," ",currList[i])
            target.append(1)
        elif currList[i] < minThreshold: # Scale Down
            #print("MinThres ",minThreshold," ",currList[i])
            target.append(-1)
        else:
            #print("Same ",currList[i])
            target.append(0)  #Remain same
    
    return target


def generatePredictions(model, maxLengthParam,containerCount,minThreshold, maxThreshold):
    
    usageArray = np.empty(shape = (0,containerCount))
    
    cnt = 0;
    
    correctOutputArray = np.empty(shape = (0,containerCount))
    
    predictionsArray = np.empty(shape = (0,containerCount))
    
    while cnt<80:
        
        predictions,usageArray = getPredictionsForOneTimeStep(model,usageArray,maxLengthParam)
        
        predictionsArray = appendToList(predictionsArray, predictions, maxLengthParam)
        
        if(usageArray.shape[0]==maxLengthParam):
    
            correctOutputsList = getRightOutputs(usageArray,minThreshold, maxThreshold)

            correctOutputArray = appendToList(correctOutputArray, correctOutputsList, maxLengthParam)
            
            print("Predictions: ",predictions)
            print("Expected Outputs: ",correctOutputsList)
            print(accuracy_score(predictions,correctOutputsList))
            
            #for i in range(usageArray.shape[1]):

                #correctOutputsForTraining = to_categorical(correctOutputs,num_classes = 3)
        
        cnt+=1
    
    
    
    

In [5]:
LSTMModel = getTrainedLSTMModel(50)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
temp = cpuinfo()
generatePredictions(LSTMModel,50,len(temp),30,70)

[ 6.2048594 15.4269942  0.        41.6213113  4.9339693]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 6.2048594 15.4269942  0.        41.6213113  4.8244155]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 6.1120891 14.5741581  0.        42.3681654  4.8244155]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 6.1120891 12.8007786  0.        39.5758704  4.7088931]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 5.3150079 12.8007786  0.        39.5758704  4.7088931]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 5.3150079 12.8007786  0.        39.5758704  4.7088931]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 5.3150079 13.3291659  0.        37.317058   4.5659639]
Predictions:  [-1, -1, -1, -1, -1]
Expected Outputs:  [-1, -1, -1, 0, -1]
0.8
[ 5.3847789 13.3291659  0.        37.317058   4.5659639

In [ ]:
mylist = np.array([cpuinfo()])
print(mylist)
print(mylist.shape)

In [ ]:
mylist2 = np.insert(mylist,0, np.array([cpuinfo()]), axis = 0)
print(mylist2)

In [ ]:
mylist = np.append(np.empty(shape = (0,5)), np.array([cpuinfo()]), axis = 0)
print(mylist)
print(mylist.shape)

In [ ]:
mylist[:,1]

In [ ]:
a = [1,2,3,4]
a[1:]

In [ ]:
b = np.random.randint(1,100,size = (10,5))
b

In [ ]:
np.expand_dims(np.expand_dims(b[:,1],axis = 1),axis = 0).shape
#b[:,1]